# Import the packages

In [175]:
import pandas as pd
import numpy as np
import ujson as json
import time, datetime
import torch, nltk, re, random
from transformers import BertTokenizer
from nltk.stem.porter import *
from torch.utils.data import TensorDataset, random_split, DataLoader, RandomSampler, SequentialSampler


import warnings
warnings.filterwarnings("ignore", category=UserWarning) 


# Load/Prepare Datasets

In [132]:
pd.set_option('display.max_columns', None)
stemmer = PorterStemmer()
# Load the twitter DS - 24784 Entries
df = pd.read_csv('datasets/twitter.csv') # https://github.com/t-davidson/hate-speech-and-offensive-language
print('Tweet categories: {}'.format(df['class'].unique())) # 0 - hate speech | 1 - offensive language | 2 - neither

def preprocess(text_string): # Ref: https://github.com/t-davidson/hate-speech-and-offensive-language/blob/master/classifier/final_classifier.ipynb
    """ Accepts a text string and replaces:
        1) urls with URLHERE
        2) lots of whitespace with one instance
        3) mentions with MENTIONHERE
        Get standardized counts of urls and mentions w/o caring about specific people mentioned
    @ retrun 
       List of stemmed words in a sentence
    """
    space_pattern = '\s+'
    giant_url_regex = ('http[s]?://(?:[a-zA-Z]|[0-9]|[$-_@.&+]|'
        '[!*\(\),]|(?:%[0-9a-fA-F][0-9a-fA-F]))+')
    mention_regex = '@[\w\-]+'
    parsed_text = re.sub(space_pattern, ' ', text_string)
    parsed_text = re.sub(giant_url_regex, '', parsed_text)
    parsed_text = re.sub(mention_regex, '', parsed_text)

    parsed_text = " ".join(re.split("[.,!?:\t\n\"]", parsed_text.lower()))  # Doc: https://docs.python.org/3/library/re.html?highlight=split#re.split
    stemmed_text = [stemmer.stem(t) for t in parsed_text.split()]
    # stemmed_text = " ".join(stemmed_text)

    return stemmed_text

# Apply preprocess to dataframe
df = pd.concat([df['tweet'], df['class'].astype(int)], axis = 1)
print('Total words (before preprocess): ' + str(len(df) * np.sum([len(sent) for sent in df['tweet']])))
df = pd.concat([df['tweet'].apply(preprocess), df['class'].astype(int)], axis = 1)
print('Total words (after  preprocess): ' + str(len(df) * np.sum([len(sent) for sent in df['tweet']])))

# None hateful tweets (some may contain "sensitive" words)
df_subclass = df.loc[df['class']==2]
print('---- Number of [neutral] tweets: {} ({}%)'.format(len(df_subclass), round(len(df_subclass)/len(df), 4) * 100))
print(df_subclass['tweet'].head(5))

# Offensive tweets - not-quite hateful
df_subclass = df.loc[df['class']==1]
print('---- Number of [offensive] tweets: {} ({}%)'.format(len(df_subclass), round(len(df_subclass)/len(df), 3) * 100))
print(df_subclass['tweet'].head(5))

# Hateful tweets
df_subclass = df.loc[df['class']==0]
print('---- Number of [HATEFUL] tweets: {} ({}%)'.format(len(df_subclass), round(len(df_subclass)/len(df), 3) * 100))
print(df_subclass['tweet'].head(5))

Tweet categories: [2 1 0]
Total words (before preprocess): 52474582446
Total words (after  preprocess): 8178588264
---- Number of [neutral] tweets: 4163 (16.8%)
0     [rt, as, a, woman, you, shouldn't, complain, a...
40    [momma, said, no, pussi, cat, insid, my, doghous]
63    [-simplyaddictedtoguy, woof, woof, hot, scalli...
66                         [woof, woof, and, hot, sole]
67    [lemmi, eat, a, oreo, &amp;, do, these, dish, ...
Name: tweet, dtype: object
---- Number of [offensive] tweets: 19190 (77.4%)
1    [rt, boy, dat, cold, tyga, dwn, bad, for, cuff...
2    [rt, dawg, rt, you, ever, fuck, a, bitch, and,...
3                     [rt, she, look, like, a, tranni]
4    [rt, the, shit, you, hear, about, me, might, b...
5    [the, shit, just, blow, me, claim, you, so, fa...
Name: tweet, dtype: object
---- Number of [HATEFUL] tweets: 1430 (5.800000000000001%)
85                                       [queer, gaywad]
89     [alsarabsss, he, a, beaner, smh, you, can, tel...
110    [

In [146]:
sentences = df['tweet'].values
labels = df['class'].values
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased', do_lower_case=True)

# Tokenize sentences. This is an example (i.e. result not used)
max_len = 0
for i, sent in enumerate(sentences):
    # Tokenize the text and add `[CLS]` and `[SEP]` tokens. Ref: https://mccormickml.com/2019/07/22/BERT-fine-tuning/
    input_ids = tokenizer.encode(sent, add_special_tokens=True)
    # Update the maximum sentence length.
    max_len = max(max_len, len(input_ids))
    
    if i < 10: # Visualize first 10 sentences
        print('NumWords: {} | NumTokens: {}'.format(len(sent), len(input_ids)))
        print('\tWords:  {}...\n\tTokens: {}...'.format(sent[:10], input_ids[:10]))
        # NOTE: NumTokens = NumWords + 2, because of [CLS] nad [SEP] tokens

print('---- Max sentence length: {} ----'.format(max_len)) # 36 NOTE: We'll set max_length to 48

NumWords: 23 | NumTokens: 25
	Words:  ['rt', 'as', 'a', 'woman', 'you', "shouldn't", 'complain', 'about', 'clean', 'up']...
	Tokens: [101, 19387, 2004, 1037, 2450, 2017, 100, 17612, 2055, 4550]...
NumWords: 15 | NumTokens: 17
	Words:  ['rt', 'boy', 'dat', 'cold', 'tyga', 'dwn', 'bad', 'for', 'cuffin', 'dat']...
	Tokens: [101, 19387, 2879, 23755, 3147, 100, 100, 2919, 2005, 100]...
NumWords: 18 | NumTokens: 20
	Words:  ['rt', 'dawg', 'rt', 'you', 'ever', 'fuck', 'a', 'bitch', 'and', 'she']...
	Tokens: [101, 19387, 100, 19387, 2017, 2412, 6616, 1037, 7743, 1998]...
NumWords: 6 | NumTokens: 8
	Words:  ['rt', 'she', 'look', 'like', 'a', 'tranni']...
	Tokens: [101, 19387, 2016, 2298, 2066, 1037, 100, 102]...
NumWords: 24 | NumTokens: 26
	Words:  ['rt', 'the', 'shit', 'you', 'hear', 'about', 'me', 'might', 'be', 'true']...
	Tokens: [101, 19387, 1996, 4485, 2017, 2963, 2055, 2033, 2453, 2022]...
NumWords: 19 | NumTokens: 21
	Words:  ['the', 'shit', 'just', 'blow', 'me', 'claim', 'you', 'so', 

In [161]:
# Tokenize all of the sentences and map the tokens to thier word IDs.
input_ids = []
attention_masks = []

for sent in sentences: # `encode_plus` will:
    # 1. Tokenize the sentence.
    # 2. Prepend the `[CLS]` token to the start.
    # 3. Append the `[SEP]` token to the end.
    # 4. Map tokens to their IDs.
    # 5. Pad or truncate the sentence to `max_length`
    # 6. Create attention masks for [PAD] tokens.
    encoded_dict = tokenizer.encode_plus(
                        sent,                      # Sentence to encode.
                        add_special_tokens = True, # Add '[CLS]' and '[SEP]'
                        max_length = 48,           # Pad & truncate all sentences.
                        pad_to_max_length = True,
                        return_attention_mask = True,   # Construct attn. masks.
                        return_tensors = 'pt',     # Return pytorch tensors.
                   )
    
    # Add the encoded sentence to the list.    
    input_ids.append(encoded_dict['input_ids'])
    
    # And its attention mask (simply differentiates padding from non-padding).
    attention_masks.append(encoded_dict['attention_mask'])

# Convert the lists into tensors.
input_ids = torch.cat(input_ids, dim=0)
attention_masks = torch.cat(attention_masks, dim=0)
labels = torch.tensor(labels)

# Print sentence 0, now as a list of IDs.
print('Words: ', sentences[0])
print('Tokens:', input_ids[0].numpy())


Words:  ['rt', 'as', 'a', 'woman', 'you', "shouldn't", 'complain', 'about', 'clean', 'up', 'your', 'hous', '&amp;', 'as', 'a', 'man', 'you', 'should', 'alway', 'take', 'the', 'trash', 'out']
Tokens: [  101 19387  2004  1037  2450  2017   100 17612  2055  4550  2039  2115
   100   100  2004  1037  2158  2017  2323   100  2202  1996 11669  2041
   102     0     0     0     0     0     0     0     0     0     0     0
     0     0     0     0     0     0     0     0     0     0     0     0]


In [171]:
# Training & Validation Split

# Combine the training inputs into a TensorDataset.
dataset = TensorDataset(input_ids, attention_masks, labels)

train_size, val_size = int(0.7 * len(dataset)), int(0.15 * len(dataset))
test_size = len(dataset) - train_size - val_size

# Split: Train 70% | Val 15% | Test 15%
print("Total Samples: {}\n\tTrain {} | Val {} | Test {}".format(len(dataset), train_size, val_size, test_size))
train_dataset, val_dataset, test_dataset = random_split(dataset, [train_size, val_size, test_size],  
                                    generator=torch.Generator().manual_seed(42))

BATCH_SIZE = 32 # For fine-tuning BERT, the authors recommend a batch size of 16 or 32.

# Create the DataLoaders for our training and validation sets.
train_dataloader = DataLoader(train_dataset,  
            sampler = RandomSampler(train_dataset), # Select batches randomly
            batch_size = BATCH_SIZE)

validation_dataloader = DataLoader(val_dataset, 
            sampler = SequentialSampler(val_dataset), # Select batches sequentially.
            batch_size = BATCH_SIZE)

test_dataloader = DataLoader(test_dataset, 
            sampler = SequentialSampler(test_dataset),
            batch_size = BATCH_SIZE)


Total Samples: 24783
	Train 17348 | Val 3717 | Test 3718


## The BERT Model

In [172]:
# Define Model
from transformers import BertForSequenceClassification, AdamW, BertConfig, get_linear_schedule_with_warmup

# Load BertForSequenceClassification, the pretrained BERT model with a single linear classification layer on top. 
model = BertForSequenceClassification.from_pretrained(
    "bert-base-uncased", # Use the 12-layer BERT model, with an uncased vocab.
    num_labels = 3, # 3 Classes - [0 Hateful, 1 Offensive, 2 Neutral]
    output_attentions = False, # Whether the model returns attentions weights.
    output_hidden_states = False, # Whether the model returns all hidden-states.
)

# model.cuda() # TODO: Uncomment this when have GPU access

# NOTE:Optim from huggingface (not pytorch). 'W' stands for 'Weight Decay fix"
optimizer = AdamW(model.parameters(),
                  lr = 2e-5, # args.learning_rate - default is 5e-5, our notebook had 2e-5
                  eps = 1e-8 # args.adam_epsilon  - default is 1e-8.
                )

epochs = 4 # The BERT authors recommend between 2 and 4. 

# Total number of training steps is [number of batches] x [number of epochs]. 
# (Note that this is not the same as the number of training samples).
total_steps = len(train_dataloader) * epochs

# Create the learning rate scheduler.
scheduler = get_linear_schedule_with_warmup(optimizer, 
                                            num_warmup_steps = 0, # Default value in run_glue.py
                                            num_training_steps = total_steps)

Downloading: 100%|██████████| 420M/420M [00:08<00:00, 50.2MB/s] 
Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForSequenceClassification: ['cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.decoder.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.bias', 'cls.predictions.transform.dense.bias', 'cls.seq_relationship.weight']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenc

## Training 

In [ ]:
# Function to calculate the accuracy of our predictions vs labels
def flat_accuracy(preds, labels):
    pred_flat = np.argmax(preds, axis=1).flatten()
    labels_flat = labels.flatten()
    return np.sum(pred_flat == labels_flat) / len(labels_flat)

def format_time(elapsed):
    elapsed_rounded = int(round((elapsed))) # Round to the nearest second.
    return str(datetime.timedelta(seconds=elapsed_rounded)) # Format as hh:mm:ss


# Ref: https://github.com/huggingface/transformers/blob/5bfcd0485ece086ebcbed2d008813037968a9e58/examples/run_glue.py#L128

# Set the seed value all over the place to make this reproducible.
seed_val = 42

random.seed(seed_val)
np.random.seed(seed_val)
torch.manual_seed(seed_val)
torch.cuda.manual_seed_all(seed_val)

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

training_stats = [] # Store training and validation loss, validation accuracy, and timings.
total_t0 = time.time() # Measure the total training time for the whole run.

# For each epoch...
for epoch_i in range(0, epochs):

    print("")
    print('======== Epoch {:} / {:} ========'.format(epoch_i + 1, epochs))
    print('Training...')

    # Measure how long the training epoch takes.
    t0 = time.time()

    # Reset the total loss for this epoch.
    total_train_loss = 0

    # Put the model into training mode. Don't be mislead--the call to 
    # `train` just changes the *mode*, it doesn't *perform* the training.
    # `dropout` and `batchnorm` layers behave differently during training
    # vs. test (source: https://stackoverflow.com/questions/51433378/what-does-model-train-do-in-pytorch)
    model.train()

    # For each batch of training data...
    for step, batch in enumerate(train_dataloader):

        # Progress update every 40 batches.
        if step % 40 == 0 and not step == 0:
            # Calculate elapsed time in minutes.
            elapsed = format_time(time.time() - t0)
            
            # Report progress.
            print('  Batch {:>5,}  of  {:>5,}.    Elapsed: {:}.'.format(step, len(train_dataloader), elapsed))

        # Unpack this training batch from our dataloader. `batch` contains three pytorch tensors:
        #   [0]: input ids 
        #   [1]: attention masks
        #   [2]: labels 
        b_input_ids = batch[0].to(device)
        b_input_mask = batch[1].to(device)
        b_labels = batch[2].to(device)

        # Always clear any previously calculated gradients before performing a
        # backward pass. PyTorch doesn't do this automatically because 
        # accumulating the gradients is "convenient while training RNNs". 
        # (source: https://stackoverflow.com/questions/48001598/why-do-we-need-to-call-zero-grad-in-pytorch)
        model.zero_grad()        

        # Perform a forward pass (evaluate the model on this training batch).
        # The documentation for this `model` function is here: 
        # https://huggingface.co/transformers/v2.2.0/model_doc/bert.html#transformers.BertForSequenceClassification
        # It returns different numbers of parameters depending on what arguments
        # arge given and what flags are set. For our useage here, it returns
        # the loss (because we provided labels) and the "logits"--the model
        # outputs prior to activation.
        loss, logits = model(b_input_ids, 
                             token_type_ids=None, 
                             attention_mask=b_input_mask, 
                             labels=b_labels)

        # Accumulate the training loss over all of the batches so that we can
        # calculate the average loss at the end. `loss` is a Tensor containing a
        # single value; the `.item()` function just returns the Python value 
        # from the tensor.
        total_train_loss += loss.item()

        # Perform a backward pass to calculate the gradients.
        loss.backward()

        # Clip the norm of the gradients to 1.0.
        # This is to help prevent the "exploding gradients" problem.
        torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)

        # Update parameters and take a step using the computed gradient.
        # The optimizer dictates the "update rule"--how the parameters are
        # modified based on their gradients, the learning rate, etc.
        optimizer.step()

        # Update the learning rate.
        scheduler.step()

    # Calculate the average loss over all of the batches.
    avg_train_loss = total_train_loss / len(train_dataloader)            
    
    # Measure how long this epoch took.
    training_time = format_time(time.time() - t0)

    print("")
    print("  Average training loss: {0:.2f}".format(avg_train_loss))
    print("  Training epcoh took: {:}".format(training_time))
        

    print("\n Validation...")

    t0 = time.time()

    # Put the model in evaluation mode--the dropout layers behave differently
    # during evaluation.
    model.eval()

    # Tracking variables 
    total_eval_accuracy = 0
    total_eval_loss = 0
    nb_eval_steps = 0

    # Evaluate data for one epoch
    for batch in validation_dataloader:
        
        # Unpack this training batch from our dataloader. 
        #
        # As we unpack the batch, we'll also copy each tensor to the GPU using 
        # the `to` method.
        #
        # `batch` contains three pytorch tensors:
        #   [0]: input ids 
        #   [1]: attention masks
        #   [2]: labels 
        b_input_ids = batch[0].to(device)
        b_input_mask = batch[1].to(device)
        b_labels = batch[2].to(device)
        
        # Tell pytorch not to bother with constructing the compute graph during
        # the forward pass, since this is only needed for backprop (training).
        with torch.no_grad():        

            # Forward pass, calculate logit predictions.
            # token_type_ids is the same as the "segment ids", which 
            # differentiates sentence 1 and 2 in 2-sentence tasks.
            # The documentation for this `model` function is here: 
            # https://huggingface.co/transformers/v2.2.0/model_doc/bert.html#transformers.BertForSequenceClassification
            # Get the "logits" output by the model. The "logits" are the output
            # values prior to applying an activation function like the softmax.
            (loss, logits) = model(b_input_ids, 
                                   token_type_ids=None, 
                                   attention_mask=b_input_mask,
                                   labels=b_labels)
            
        # Accumulate the validation loss.
        total_eval_loss += loss.item()

        # Move logits and labels to CPU
        logits = logits.detach().cpu().numpy()
        label_ids = b_labels.to('cpu').numpy()

        # Calculate the accuracy for this batch of test sentences, and
        # accumulate it over all batches.
        total_eval_accuracy += flat_accuracy(logits, label_ids)
        

    # Report the final accuracy for this validation run.
    avg_val_accuracy = total_eval_accuracy / len(validation_dataloader)
    print("  Accuracy: {0:.2f}".format(avg_val_accuracy))

    # Calculate the average loss over all of the batches.
    avg_val_loss = total_eval_loss / len(validation_dataloader)
    
    # Measure how long the validation run took.
    validation_time = format_time(time.time() - t0)
    
    print("  Validation Loss: {0:.2f}".format(avg_val_loss))
    print("  Validation took: {:}".format(validation_time))

    # Record all statistics from this epoch.
    training_stats.append(
        {
            'epoch': epoch_i + 1,
            'Training Loss': avg_train_loss,
            'Valid. Loss': avg_val_loss,
            'Valid. Accur.': avg_val_accuracy,
            'Training Time': training_time,
            'Validation Time': validation_time
        }
    )

print("")
print("Training complete!")

print("Total training took {:} (h:mm:ss)".format(format_time(time.time()-total_t0)))

In [173]:

# pd.set_option('display.max_rows', None)
# parler = map(json.loads, open('./datasets/parler_pretrain.ndjson'))
# parler = pd.DataFrame.from_records(parler)
# parler = parler.loc[(parler['body']!='')] # Filter empty body. Final len: 636420 outof 1094836 (58.12925406179556%)
# parler = parler[['body']]
# print(len(parler))
# parler.head(100)